In [ ]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import scipy.misc
from glob import glob
import numpy as np
import random 
import shutil 
import keras
import matplotlib.pyplot as plt

base_path = os.path.abspath("../")
data_name = "cxr"


sys.path.append(base_path)
import helpers

In [ ]:
def load_img(path):
    # read image
    img = scipy.misc.imread(path, mode='RGB').astype(np.float)
    #  resize
    img = scipy.misc.imresize(img, (256,256))
    
    # rescale to [-1, 1]
    #img = img/127.5 - 1.
    # rescale to [0, 1]
    #img = img.astype(np.float32)
    #img /= 255.0
    
    return img

In [ ]:
# Image paths
cov19 = os.path.join(base_path, "dataset/{}/test/cov19".format(data_name))
normal = os.path.join(base_path, "dataset/{}/test/normal".format(data_name))

print(cov19, normal)

In [ ]:
cov19_path = [os.path.join(cov19, x) for x in os.listdir(cov19)]
normal_path = [os.path.join(normal, x) for x in os.listdir(normal)]

len(cov19_path), len(normal_path)

In [ ]:
random.shuffle(cov19_path)
random.shuffle(normal_path)

cov19_path = cov19_path[:]
normal_path = normal_path[:46]

In [ ]:
covid19_images=np.array([load_img(path) for path in cov19_path])
covid19_images.shape

In [ ]:
normal_images=np.array([load_img(path) for path in normal_path])
normal_images.shape

In [ ]:
#---------------------------
# cxr
# 0 - cov
# 1 - normal
# 2 - pneu
#---------------------------
# cxr_normalvcovid
# 0 - cov19
# 1 - normal
#---------------------------
# cxr_pneucovid
# 0 - cov19
# 1 - pneu
#---------------------------

In [ ]:
# Normal vs Covid
#normal_labels = np.array([1 for _ in range(len(normal_images))])
#cov19_labels = np.array([0 for _ in range(len(covid19_images))])

normal_labels = np.array([0 for _ in range(len(normal_images))])
cov19_labels = np.array([1 for _ in range(len(covid19_images))])

print(cov19_labels.shape, normal_labels.shape)

x_all = np.concatenate((normal_images, covid19_images), axis=0)
y_all = np.concatenate((normal_labels, cov19_labels), axis=0)
y_all = keras.utils.to_categorical(y_all, 2)

print(x_all.shape, y_all.shape)

In [ ]:
# Standardize
x_all = x_all.astype('float32')
x_all /= 255

In [ ]:
from keras.models import load_model

EXPERIMENT_NAME =  "cxr_normalvcovid+g1g2" 
LOG_PATH = os.path.join(base_path, "models", EXPERIMENT_NAME)
model = None
model = load_model("{}/{}.h5".format(LOG_PATH, EXPERIMENT_NAME), compile = False)
model.summary()

In [ ]:
plt.imshow(x_all[0])

In [ ]:
#y_pred = []
#for img in x_all:
#    img = np.expand_dims(img, axis=0)
#    prob = model.predict(img, verbose=1)
#    y_pred.append(prob)
    
#y_pred = np.array(y_pred)
#y_pred.shape

In [ ]:
# Make predictions using trained model
y_pred = model.predict(x_all, verbose=1)
print("Predictions: ", y_pred.shape)

In [ ]:
# Convert ground truth to column values
y_test_flat = np.argmax(y_all, axis=1)
print("After flattening ground truth: ", y_test_flat.shape)

# Get labels from predictions
y_pred_flat = np.array([np.argmax(pred) for pred in y_pred]) # y_pred[1] -> probability for class 1 
print("Binarize probability values: ", y_pred_flat.shape)

assert y_pred_flat.shape == y_test_flat.shape, "Shape mismatch!"

In [ ]:
y_test_flat

In [ ]:
y_pred_flat

In [ ]:
def deprocess_image(x):
    """Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    """
    x = x.copy()
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'th': #keras.backend.image_data_format()
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def normalize(x):
    """Utility function to normalize a tensor by its L2 norm"""
    return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)


def get_heatmap(gc):
  """Convert 2D heatmap to 3D for plotting"""
  # Get the color map
  cm = plt.get_cmap('jet')
  # Apply the colormap like a function to any array:
  gc3 = cm(gc)
  gc3 = gc3[:, :, :3].astype('float32') 
  return gc3

def grad_cam(input_model, image, cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize if necessary
    # grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (256, 256), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam


In [ ]:
from tqdm import tqdm
import keras.backend as K
import cv2

x_test = x_all

missclass_1 = []

for i in tqdm(range(len(y_test_flat))):
  # if predicted is 0 and actual is 1
  if y_pred_flat[i] == 1 and y_test_flat[i] == 1:
    missclass_1.append(x_test[i])

print(len(missclass_1))
# take first 10 missclassified sampels
missclass_1 = np.array(missclass_1[:5])

print("Number of missclassified samples:", missclass_1.shape)


missclass_1_gcam = []

for ms in tqdm(missclass_1):
  ms = np.expand_dims(ms, axis=0)
  # grad cam
  gc = grad_cam(model, ms, -1, 'block5_conv3')
  # convert to 3D
  gc = get_heatmap(gc)
  missclass_1_gcam.append(gc)

missclass_1_gcam = np.array(missclass_1_gcam)

print("Number of missclassified gradcam samples:", missclass_1_gcam.shape)

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

results = np.concatenate((missclass_1, missclass_1_gcam), axis=0)

x = results
a, b = 2, 5
x = np.reshape(x, (a, b, 256, 256, 3))

test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')

plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig("{}/models/{}/true_covid.pdf".format(base_path, EXPERIMENT_NAME), dpi=300)
plt.show()